In [1]:
# This cell loads local isearch via selenium in Chrome
from selenium import webdriver
driver = webdriver.Chrome(executable_path=r"C:\Users\tbutterf\chromedriver.exe")
driver.get("https://spa.asu.edu/admin/content/isearch");

In [2]:
# PRIMARY AFFILIATION SWITCHER
# This tool loops through all iSearch Profiles in Drupal and overrides the primary affiliation to match what
# exists in Peoplesoft. This allows the Views display to show the correct title and department.
# Requires a spreadsheet with 2 columns in this order: Name, Department

# from IPython.core.debugger import set_trace
# set_trace()
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
xldata = pd.read_excel('spa-centers.xlsx')
rows = driver.find_elements_by_xpath('//*[@id="views-form-isearch-admin-admin-list"]/div/table/tbody/tr')

for i in range(2,len(rows)+1):
    st = '//*[@id="views-form-isearch-admin-admin-list"]/div/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = driver.find_element_by_xpath(st+'/td[2]/a')
    nameval = name.text
    unit = xldata[xldata['Name']==nameval]['Department'].values
    try:
        match = len(unit[0])
    except:
        continue
    if match > 0:
        name.click()
        edit = driver.find_element_by_xpath('//*[@id="tabs"]/ul/li[2]/a')
        edit.click()
        selectbox = Select(driver.find_element_by_xpath(
                '//*[@id="edit-field-isearch-local-primary-aff-und"]'))
        none = driver.find_element_by_xpath(
            '//*[@id="edit-field-isearch-local-primary-aff-und"]/option[contains(text(),"None")]').text
        selectedaff = selectbox.first_selected_option.text
        if selectedaff.find(unit[0])!= -1:
            print(str(i)+'. Already overridden: '+nameval+', '+selectedaff)
            driver.get("https://spa.asu.edu/admin/content/isearch")
            time.sleep(1)
        else:
            try:
                unitaff = driver.find_element_by_xpath('//*[@id="edit-field-isearch-local-primary-aff-und"]'\
                                                       '/option[contains(text(), "'+unit[0]+'")]').text
                selectbox.select_by_visible_text(unitaff)
                selectedaff = selectbox.first_selected_option.text
                save = driver.find_element_by_xpath('//*[@id="edit-submit"]')
                save.click()
                print(str(i)+'. Success: '+nameval+', '+selectedaff)
            except:
                print(str(i)+'. Unit not available: '+nameval+', '+unit[0])
                pass
            try: 
                error = driver.find_element_by_xpath('//*[@id="page-title"][contains(text(),"Error")]').text
                if (len(error)>0):
                    #driver.get("https://spa.asu.edu/admin/content/isearch")
                    driver.back(2)
                    time.sleep(1)
                    print(str(i)+'. Error: '+nameval)
                    
            except NoSuchElementException:
                driver.get("https://spa.asu.edu/admin/content/isearch")
                time.sleep(1)
#         except NoSuchElementException:
#             print(str(i)+'. Error: '+nameval+'. Missing unit affiliation ('+unit+')')
#             driver.get("https://publicservice.asu.edu/admin/content/isearch")
#             time.sleep(1)
#             
print("Done!")

5. Success: Julia Bausch,   Morrison Institute for Public Policy (Postdoctoral Research Assistant) 
28. Success: Susan Craig,   Morrison Institute for Public Policy (Research Analyst) 
32. Success: Timothy Dahlstrom,   Bob Ramsey Executive Education (Faculty Associate) 
60. Success: Linda Hess,   Bob Ramsey Executive Education (Senior Program Manager) 
68. Success: Dan Hunting,   Morrison Institute for Public Policy (Management Research Analyst) 
83. Success: Melissa Kovacs,   Morrison Institute for Public Policy (Associate Director) 
93. Success: Michele Lynch,   Morrison Institute for Public Policy (Administrative Assistant) 
94. Success: Diane Mallory,   Bob Ramsey Executive Education (Program Manager) 
100. Success: Lesley Michalegko,   Center for Science, Technology & Environmental Policy Studies (C-STEPS) (Project Coordinator) 
106. Success: Mary O'Brien,   Center for Organizational Research & Design (CORD) (Administrative Specialist) 
110. Success: Sarah Porter,   Morrison Insti

In [ ]:
# UNPUBLISH TOOL
# This tool loops through all iSearch Profiles in Drupal and unpublishes matching profiles based on 
# a pre-filtered Excel Spreadsheet with 1 column: Name. The people in this sheet will be unpublished.

# from IPython.core.debugger import set_trace
# set_trace()
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
xldata = pd.read_excel('socialwork-to-be-unpublished.xlsx')
rows = driver.find_elements_by_xpath('//*[@id="views-form-isearch-admin-admin-list"]/div/table/tbody/tr')
unpublishlist = []
for i in range(2,len(rows)+1):
    st = '//*[@id="views-form-isearch-admin-admin-list"]/div/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = driver.find_element_by_xpath(st+'/td[2]/a')
    box = driver.find_element_by_xpath(st+'/td[1]').find_element_by_css_selector('input.form-checkbox')
    nameval = name.text
    unit = xldata[xldata['Name']==nameval]['Name'].values
    try:
        match = len(unit)
    except:
        continue
    if match > 0:
        box.click()
        unpublishlist.append(name)
operations = Select(driver.find_element_by_xpath('//*[@id="edit-operation"]'))
operations.select_by_visible_text('Unpublish content')
execute = driver.find_element_by_xpath('//*[@id="edit-submit--2"]')
execute.click()
time.sleep(1)
confirm = driver.find_element_by_xpath('//*[@id="edit-submit"]')
confirm.click()
print("Unpublish list:",*unpublishlist, sep='\n')
print("Done!")